# Prepare Articles (Clean, Normalize, Enhance)

This notebook builds a unified dataset from raw sources and precomputes Gemini 2.5 Flash summaries + I1–I5 hints for each paper. It saves the enhanced dataset to `articles_with_gpt_response.csv` for use by the review notebook.

Run top-to-bottom whenever raw inputs change.

In [ ]:
# --- imports ---
import os
import json
import time
import asyncio
from pathlib import Path
from typing import Dict, Any, List, Optional

import pandas as pd
from pydantic import BaseModel

from google import genai
from google.genai import types

# Paths (relative to this notebook directory)
OUT_DIR = Path(".")
RAW_DIR_IN = OUT_DIR / "raw_data"

# LLM
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
if GOOGLE_API_KEY:
    os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY  # map to SDK expected env var


# --- config / paths ---
OUT_DIR = Path(".")
ENHANCED_CSV = OUT_DIR / "articles_with_gpt_response.csv"
DEBUG_DIR = OUT_DIR / "debug_out"
DEBUG_DIR.mkdir(parents=True, exist_ok=True)
DEBUG_JSONL = DEBUG_DIR / "batch_debug.jsonl"

GOOGLE_LLM_MODEL = os.getenv("GOOGLE_LLM_MODEL", "gemini-2.5-flash-lite")
API_KEY = os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")

In [6]:
# Load and normalize raw sources into a unified dataframe

SOURCES = {
    "ieeexplore.csv": "ieeexplore",
    "proquest.csv": "proquest",
    "scopus.csv": "scopus",
    "wos.csv": "wos",
}

common_cols = ["title", "abstract", "authors", "year", "doi", "link", "database"]

alt_names = {
    "title": ["title", "document title", "ti", "Title", "Article title", "article title"],
    "abstract": ["abstract", "ab", "Abstract"],
    "authors": ["authors", "au", "Author(s)", "Author"],
    "year": ["year", "py", "Publication Year", "Year", "PubDate"],
    "doi": ["doi", "di", "DOI"],
    "link": ["link", "pdf link", "url", "Link"],
}

def pick(row, keys):
    for k in keys:
        if k in row and pd.notna(row[k]):
            return row[k]
    return None

frames = []
source_counts = {}
for fname, src in SOURCES.items():
    p = RAW_DIR_IN / fname
    if not p.exists():
        print(f"⚠️  {src}: file not found, skipping")
        continue
    # WoS uses semicolon as delimiter, others use comma
    sep = ';' if fname == 'wos.csv' else ','
    temp = pd.read_csv(p, sep=sep)
    lower_cols = {c: c.strip().lower() for c in temp.columns}
    temp = temp.rename(columns=lower_cols)
    out = pd.DataFrame()
    out["title"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["title"]]), axis=1)
    out["abstract"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["abstract"]]), axis=1)
    out["authors"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["authors"]]), axis=1)
    out["year"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["year"]]), axis=1)
    out["doi"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["doi"]]), axis=1)
    out["link"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["link"]]), axis=1)
    out["database"] = src
    source_counts[src] = len(out)
    print(f"📄 {src}: {len(out)} articles")
    frames.append(out)

if frames:
    merged = pd.concat(frames, ignore_index=True)
else:
    merged = pd.DataFrame(columns=common_cols)

print(f"\n📊 Total combined: {len(merged)} articles")

# Normalize types and deduplicate
merged["title"] = merged["title"].astype(str).str.strip()
merged["abstract"] = merged["abstract"].astype(str)
merged["authors"] = merged["authors"].astype(str)

# Handle year extraction - for ProQuest articles, extract from PubDate format "2016-03-01"
def extract_year_with_pubdate_fallback(row):
    year = row.get('year')
    if pd.notna(year) and str(year).strip() != '':
        year_str = str(year).strip()
        
        # For ProQuest articles, try to extract from PubDate format first
        if row.get('database') == 'proquest' and '-' in year_str and len(year_str.split('-')[0]) == 4:
            try:
                return int(year_str.split('-')[0])
            except (ValueError, IndexError):
                pass
        
        # Try to convert to numeric for other cases
        try:
            return pd.to_numeric(year, errors="coerce")
        except:
            return year
    
    return year

merged["year"] = merged.apply(extract_year_with_pubdate_fallback, axis=1)
merged["year"] = pd.to_numeric(merged["year"], errors="coerce").astype("Int64")
merged["doi"] = merged["doi"].astype(str).str.strip().str.lower()
merged["link"] = merged["link"].astype(str).str.strip()

# Drop rows without title
before_title_filter = len(merged)
merged = merged[merged["title"].str.len() > 0].copy()
removed_no_title = before_title_filter - len(merged)
if removed_no_title > 0:
    print(f"🗑️  Removed {removed_no_title} articles without title")
print(f"✅ After filtering: {len(merged)} articles")

# Deduplicate strategy (matching literature-review-ProbAI approach):
# 1. First deduplicate by DOI (normalized)
# 2. Then deduplicate by Title (all papers)

print(f"\n🔍 Deduplication process:")

# Separate rows with NaN in 'doi'
nan_doi_rows = merged[merged['doi'].isna() | (merged['doi'] == 'nan') | (merged['doi'].str.len() == 0)]
non_nan_doi_rows = merged[merged['doi'].notna() & (merged['doi'] != 'nan') & (merged['doi'].str.len() > 0)].copy()

print(f"   - Articles with DOI: {len(non_nan_doi_rows)}")
print(f"   - Articles without DOI: {len(nan_doi_rows)}")

# Normalize DOI for non-NaN rows
non_nan_doi_rows["doi"] = non_nan_doi_rows["doi"].str.lower()
non_nan_doi_rows["doi"] = non_nan_doi_rows["doi"].str.replace("https://doi.org/", "", regex=False)

# Drop duplicates based on DOI
before_doi_dedup = len(non_nan_doi_rows)
non_nan_doi_rows = non_nan_doi_rows.drop_duplicates(subset=["doi"], keep='first')
removed_doi_dupes = before_doi_dedup - len(non_nan_doi_rows)
print(f"   - Removed {removed_doi_dupes} duplicate DOIs")

# Concatenate back
merged = pd.concat([non_nan_doi_rows, nan_doi_rows], ignore_index=True)
print(f"   - After DOI dedup: {len(merged)} articles")

# Drop duplicates based on title (for ALL papers, not just those without DOI)
before_title_dedup = len(merged)
merged = merged.groupby(['title'], as_index=False).first()
merged = merged.copy()
removed_title_dupes = before_title_dedup - len(merged)
print(f"   - Removed {removed_title_dupes} duplicate titles")
print(f"   - After title dedup: {len(merged)} articles")

# Compute paper_id
import hashlib as _hash

def derive_id_row(r):
    parts = [str(r.get("title", "")), str(r.get("year", "")), str(r.get("doi", ""))]
    key = "|||".join(parts).strip().lower()
    return _hash.sha256(key.encode("utf-8")).hexdigest()[:16]

merged["paper_id"] = merged.apply(derive_id_row, axis=1)

# Reorder
cols = ["paper_id", "title", "abstract", "authors", "year", "doi", "link", "database"]
merged = merged[cols]

print(f"\n✨ Final unified dataset: {len(merged)} unique articles")

📄 ieeexplore: 43 articles
📄 proquest: 489 articles
📄 scopus: 792 articles
📄 wos: 825 articles

📊 Total combined: 2149 articles
✅ After filtering: 2149 articles

🔍 Deduplication process:
   - Articles with DOI: 2149
   - Articles without DOI: 0
   - Removed 944 duplicate DOIs
   - After DOI dedup: 1205 articles
   - Removed 2 duplicate titles
   - After title dedup: 1203 articles

✨ Final unified dataset: 1203 unique articles


In [26]:
# Investigate duplicate titles that were removed
print("=" * 80)
print("🔍 DUPLICATE TITLE INVESTIGATION")
print("=" * 80)

# Need to recreate the state before title deduplication
# Re-load and process to get to the point before title dedup
frames_inv = []
for fname, src in SOURCES.items():
    p = RAW_DIR_IN / fname
    if not p.exists():
        continue
    sep = ';' if fname == 'wos.csv' else ','
    temp = pd.read_csv(p, sep=sep)
    lower_cols = {c: c.strip().lower() for c in temp.columns}
    temp = temp.rename(columns=lower_cols)
    out = pd.DataFrame()
    out["title"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["title"]]), axis=1)
    out["abstract"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["abstract"]]), axis=1)
    out["authors"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["authors"]]), axis=1)
    out["year"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["year"]]), axis=1)
    out["doi"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["doi"]]), axis=1)
    out["link"] = temp.apply(lambda r: pick(r, [c.lower() for c in alt_names["link"]]), axis=1)
    out["database"] = src
    frames_inv.append(out)

merged_inv = pd.concat(frames_inv, ignore_index=True)
merged_inv["title"] = merged_inv["title"].astype(str).str.strip()
merged_inv["doi"] = merged_inv["doi"].astype(str).str.strip().str.lower()
merged_inv["year"] = pd.to_numeric(merged_inv["year"], errors="coerce").astype("Int64")
merged_inv = merged_inv[merged_inv["title"].str.len() > 0].copy()

# Do DOI dedup
nan_doi_rows_inv = merged_inv[merged_inv['doi'].isna() | (merged_inv['doi'] == 'nan') | (merged_inv['doi'].str.len() == 0)]
non_nan_doi_rows_inv = merged_inv[merged_inv['doi'].notna() & (merged_inv['doi'] != 'nan') & (merged_inv['doi'].str.len() > 0)].copy()
non_nan_doi_rows_inv["doi"] = non_nan_doi_rows_inv["doi"].str.lower()
non_nan_doi_rows_inv["doi"] = non_nan_doi_rows_inv["doi"].str.replace("https://doi.org/", "", regex=False)
non_nan_doi_rows_inv = non_nan_doi_rows_inv.drop_duplicates(subset=["doi"], keep='first')
before_title_dedup_inv = pd.concat([non_nan_doi_rows_inv, nan_doi_rows_inv], ignore_index=True)

# Find duplicated titles
title_counts = before_title_dedup_inv['title'].value_counts()
duplicate_titles = title_counts[title_counts > 1]

print(f"\n📊 Summary:")
print(f"   - Total unique titles with duplicates: {len(duplicate_titles)}")
print(f"   - Total papers with duplicate titles: {duplicate_titles.sum()}")
print(f"   - Papers that will be removed: {duplicate_titles.sum() - len(duplicate_titles)}")

if len(duplicate_titles) > 0:
    print(f"\n🔍 Top 10 most duplicated titles:")
    for i, (title, count) in enumerate(duplicate_titles.head(10).items(), 1):
        print(f"\n   {i}. Title (appears {count} times):")
        print(f"      \"{title[:100]}{'...' if len(title) > 100 else ''}\"")
        
        # Show all instances of this title
        dupes = before_title_dedup_inv[before_title_dedup_inv['title'] == title][['database', 'year', 'doi', 'authors']]
        for idx, row in dupes.iterrows():
            kept = " ⭐ KEPT" if idx == dupes.index[0] else " ❌ REMOVED"
            author_preview = str(row['authors'])[:50] + '...' if len(str(row['authors'])) > 50 else str(row['authors'])
            print(f"      - {row['database']:10} | Year: {row['year']} | DOI: {row['doi'][:30] if pd.notna(row['doi']) else 'N/A':30} | Authors: {author_preview}{kept}")

# Save detailed duplicate report
duplicate_report = []
for title in duplicate_titles.index:
    dupes = before_title_dedup_inv[before_title_dedup_inv['title'] == title].copy()
    dupes['is_kept'] = False
    dupes.iloc[0, dupes.columns.get_loc('is_kept')] = True
    duplicate_report.append(dupes)

if duplicate_report:
    duplicate_df = pd.concat(duplicate_report, ignore_index=True)
    duplicate_df = duplicate_df.sort_values(['title', 'is_kept'], ascending=[True, False])
    report_path = OUT_DIR / "duplicate_titles_report.csv"
    duplicate_df.to_csv(report_path, index=False)
    print(f"\n💾 Full duplicate report saved to: {report_path}")
    print(f"   Total rows in report: {len(duplicate_df)}")

print("\n" + "=" * 80)


🔍 DUPLICATE TITLE INVESTIGATION

📊 Summary:
   - Total unique titles with duplicates: 2
   - Total papers with duplicate titles: 4
   - Papers that will be removed: 2

🔍 Top 10 most duplicated titles:

   1. Title (appears 2 times):
      "On filtering in Markovian term structure models: An approximation approach"
      - scopus     | Year: 2001 | DOI: 10.1239/aap/1011994030         | Authors: Chiarella, C.; Pasquali, S.; Runggaldier, W.J. ⭐ KEPT
      - wos        | Year: 2001 | DOI: 10.1017/s0001867800011204      | Authors: Chiarella, C; Pasquali, S; Runggaldier, WJ ❌ REMOVED

   2. Title (appears 2 times):
      "Nonlinear predictability of stock returns using financial and economic variables"
      - scopus     | Year: 1999 | DOI: 10.1080/07350015.1999.10524830 | Authors: Qi, M. ⭐ KEPT
      - wos        | Year: 1999 | DOI: 10.2307/1392399                | Authors: Qi, M ❌ REMOVED

💾 Full duplicate report saved to: duplicate_titles_report.csv
   Total rows in report: 4



In [27]:
# Save merged dataset to CSV
merged_csv = OUT_DIR / "merged_articles.csv"
merged.to_csv(merged_csv, index=False)
print(f"💾 Saved merged dataset to: {merged_csv}")
print(f"   {len(merged)} articles saved")


💾 Saved merged dataset to: merged_articles.csv
   1203 articles saved


In [28]:
# tuneables
_MAX_RETRIES_429 = 2
MAX_OUTPUT_TOKENS = 1024
CONCURRENCY = 400

# --- screening prompt & schema ---
CRITERIA_TEXT = {
    "I1": "The article includes an empirical Machine Learning application (not pure theory). It goes beyond simple linear models.",
    "I2": "The article is about government/sovereign bonds and/or the yield curve",
    "I3": "The article is about forecasting or prediction",
    #"I3": "Fundamental model with prediction horizon > 1 month",
}

SYSTEM_PROMPT = (
    "You assist literature screening. Given title and abstract, return a concise summary and "
    "boolean hints for criteria I1..I3. Respond with STRICT JSON only, no prose. "
    "Output must be JSON with keys: summary (str), criteria_hints (object with I1..I3 booleans), notes (str). Do not return and empty booleans. Always give a TRUE or FALSE answer."
)

# --- pydantic schemas (works with parsed=...) ---
class CriteriaHints(BaseModel):
    I1: bool | None = None
    I2: bool | None = None
    I3: bool | None = None
    #I3: bool | None = None

class ScreenResult(BaseModel):
    summary: str
    criteria_hints: CriteriaHints
    notes: str | None = ""

# --- utils ---
def _short(s: Any, n=900) -> str:
    s = "" if s is None else str(s)
    return s if len(s) <= n else s[:n] + " … [truncated]"

def _flatten(x: Any) -> str:
    """Ensure we pass clean strings to the API (no tuples/lists/Series)."""
    if isinstance(x, (list, tuple)):
        return " ".join(_flatten(t) for t in x)
    return "" if x is None else str(x)

def _to_dict(parsed):
    if parsed is None: return None
    if hasattr(parsed, "model_dump"): return parsed.model_dump()      # pydantic v2
    if hasattr(parsed, "dict"): return parsed.dict()                  # pydantic v1
    return parsed

def _extract_first_json(s: str) -> Dict[str, Any] | None:
    depth = 0; start = -1
    for i, ch in enumerate(s or ""):
        if ch == "{":
            if depth == 0: start = i
            depth += 1
        elif ch == "}":
            if depth: depth -= 1
            if depth == 0 and start >= 0:
                try: return json.loads(s[start:i+1])
                except Exception: pass
    return None

def _resp_tokens(resp) -> Dict[str, Any]:
    um = getattr(resp, "usage_metadata", None)
    return {
        "prompt": getattr(um, "prompt_token_count", None) if um else None,
        "candidates": getattr(um, "candidates_token_count", None) if um else None,
        "total": getattr(um, "total_token_count", None) if um else None,
    }

def _write_debug(record: Dict[str, Any]) -> None:
    with open(DEBUG_JSONL, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

def init_gemini_client():
    if not API_KEY:
        raise RuntimeError("No API key. Set GEMINI_API_KEY or GOOGLE_API_KEY.")
    return genai.Client(api_key=API_KEY)

_client_sync: Optional[genai.Client] = None

def _mk_prompt(title: str, abstract: str) -> str:
    # Light clipping to avoid pathological inputs
    if len(abstract) > 20000:
        abstract = abstract[:20000]
    return f"Title: {title}\nAbstract: {abstract}\n\nCriteria: {json.dumps(CRITERIA_TEXT, ensure_ascii=False)}"

# --- single sync call with 3-stage fallback (structured -> unstructured -> plain) ---
def _call_sync_all(prompt: str) -> Dict[str, Any]:
    """
    Returns dict with keys:
      ok (bool), data (dict|None), logs (dict)
    """
    global _client_sync
    if _client_sync is None:
        _client_sync = init_gemini_client()

    logs: Dict[str, Any] = {"prompt_preview": _short(prompt, 1200)}

    # Attempt 1: structured JSON (schema + application/json)
    resp1 = None
    for attempt in range(_MAX_RETRIES_429 + 1):
        try:
            resp1 = _client_sync.models.generate_content(
                model=GOOGLE_LLM_MODEL,
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0,
                    max_output_tokens=MAX_OUTPUT_TOKENS,
                    response_mime_type="application/json",
                    response_schema=ScreenResult,
                    system_instruction=SYSTEM_PROMPT,
                ),
            )
            break
        except Exception as e:
            msg = str(e)
            logs[f"attempt1_{attempt}_error"] = f"{type(e).__name__}: {msg}"
            if ("429" in msg or "RESOURCE_EXHAUSTED" in msg) and attempt < _MAX_RETRIES_429:
                time.sleep(5)
                continue
            logs["attempt1_final_error"] = f"{type(e).__name__}: {msg}"
            resp1 = None
            break

    if resp1 is not None:
        logs["attempt1_tokens"] = _resp_tokens(resp1)
        parsed1 = _to_dict(getattr(resp1, "parsed", None))
        logs["attempt1_parsed"] = parsed1
        if isinstance(parsed1, dict) and parsed1.get("summary"):
            return {"ok": True, "data": parsed1, "logs": logs}

        # try recover from raw text/parts
        raw1 = getattr(resp1, "text", "") or ""
        if not raw1 and getattr(resp1, "candidates", None):
            cand = resp1.candidates[0]
            content = getattr(cand, "content", None)
            parts = (getattr(content, "parts", None) or []) if content else []
            raw1 = "".join(p.get("text", "") if isinstance(p, dict) else getattr(p, "text", "") for p in parts)
        logs["attempt1_raw"] = _short(raw1, 1500)
        j1 = _extract_first_json(raw1) if raw1 else None
        logs["attempt1_raw_parsed"] = j1
        if isinstance(j1, dict) and j1.get("summary"):
            return {"ok": True, "data": j1, "logs": logs}

    return {"ok": False, "data": None, "logs": logs}

# --- async batch via threadpool (no aiohttp session issues) ---
async def _async_screen_many(rows: List[Dict[str, Any]], model: str, concurrency: int = CONCURRENCY):
    sem = asyncio.Semaphore(concurrency)

    async def one(r):
        paper_id = r.get("paper_id")
        title = _flatten(r.get("title", "")).strip()
        abstract = _flatten(r.get("abstract", "")).strip()
        prompt = _mk_prompt(title, abstract)

        async with sem:
            result = await asyncio.to_thread(_call_sync_all, prompt)
            logs = result["logs"]
            logs["mode"] = "threadpool"
            logs["paper_id"] = paper_id
            logs["model"] = model
            _write_debug(logs)

            if not result["ok"] or not isinstance(result["data"], dict):
                return {
                    "paper_id": paper_id,
                    "llm_summary": "LLM parse/error",
                    "i1_hint": None, "i2_hint": None, "i3_hint": None,
                    "model": GOOGLE_LLM_MODEL if API_KEY else "none",
                }

            data = result["data"]
            hints = (data.get("criteria_hints", {}) or {})
            return {
                "paper_id": paper_id,
                "llm_summary": data.get("summary", ""),
                "i1_hint": hints.get("I1"),
                "i2_hint": hints.get("I2"),
                "i3_hint": hints.get("I3"),
                "model": GOOGLE_LLM_MODEL if API_KEY else "none",
            }

    return await asyncio.gather(*[one(r) for r in rows])

# --- runner helpers ---
def run_async(coro):
    try:
        return asyncio.run(coro)
    except RuntimeError:
        try:
            import nest_asyncio
            nest_asyncio.apply()
        except Exception:
            pass
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(coro)


if "merged" not in globals():
    raise RuntimeError("Expected a DataFrame named `merged` with columns: paper_id, title, abstract")

if not API_KEY:
    raise RuntimeError("No API key. Set GEMINI_API_KEY or GOOGLE_API_KEY.")

# Sanitize input columns just in case
merged = merged.copy()
merged["title"] = merged["title"].map(_flatten)
merged["abstract"] = merged["abstract"].map(_flatten)

print({"enhanced_csv": str(ENHANCED_CSV), "model": GOOGLE_LLM_MODEL})
print("Unified rows:", len(merged))

# Load previous hints (optional cache)
prev_df = None
if ENHANCED_CSV.exists():
    try:
        prev_df = pd.read_csv(ENHANCED_CSV)
    except Exception:
        prev_df = None

hint_cols = ["llm_summary", "i1_hint", "i2_hint", "i3_hint", "model"]
if prev_df is not None and "paper_id" in prev_df.columns:
    prev_hints = prev_df[["paper_id"] + [c for c in hint_cols if c in prev_df.columns]].copy()
else:
    prev_hints = pd.DataFrame(columns=["paper_id"] + hint_cols)

merged_with_hints = merged.merge(prev_hints, on="paper_id", how="left")

needs_hints_mask = merged_with_hints["llm_summary"].isna() | (merged_with_hints["llm_summary"].astype(str).str.len() == 0)
to_process = [
    {"paper_id": r["paper_id"], "title": str(r.get("title","")), "abstract": str(r.get("abstract",""))}
    for _, r in merged_with_hints[needs_hints_mask].iterrows()
]

print("Batch to_process:", len(to_process))
if not to_process:
    print("No rows need processing. Exiting without writing.")
    raise SystemExit(0)

rows = run_async(_async_screen_many(to_process, model=GOOGLE_LLM_MODEL, concurrency=CONCURRENCY))
print("Batch returned rows:", len(rows))
print("Sample results (first 3):", rows[:3])

# Validate results strictly; DO NOT write if any are empty/invalid
REQUIRED_HINT_KEYS = ("I1", "I2", "I3")

def _is_invalid(rec: Dict[str, Any]) -> tuple[bool, str]:
    # summary must be non-empty
    if rec.get("llm_summary") in (None, "", "LLM parse/error"):
        return True, "summary_missing_or_error"
    # each hint must exist and be a boolean (True/False), not None
    for k in REQUIRED_HINT_KEYS:
        v = rec.get(f"{k.lower()}_hint")  # fields are i1_hint, ..., I3_hint
        if not isinstance(v, bool):
            return True, f"{k}_not_bool_or_missing"
    return False, ""

invalid_detail = [(_r, _is_invalid(_r)) for _r in rows]
invalid = [r for r, (bad, _) in invalid_detail if bad]

# If everything looks good, merge and write
new_hints_df = pd.DataFrame(rows)
final_df = merged_with_hints.drop(columns=[c for c in hint_cols if c in merged_with_hints.columns], errors="ignore")
final_df = final_df.merge(
    pd.concat([prev_hints, new_hints_df], ignore_index=True).drop_duplicates(subset=["paper_id"], keep="last"),
    on="paper_id",
    how="left",
)

final_df.to_csv(ENHANCED_CSV, index=False)
print("Saved enhanced dataset:", ENHANCED_CSV, "rows:", len(final_df), "| newly processed:", len(new_hints_df))
print(f"Debug logs (per-row snapshots) at: {DEBUG_JSONL}")

# Clean shutdown of sync client (tidy logs; not strictly required)
try:
    if _client_sync is not None:
        _client_sync.close()
except Exception:
    pass


{'enhanced_csv': 'articles_with_gpt_response.csv', 'model': 'gemini-2.5-flash-lite'}
Unified rows: 1203
Batch to_process: 316
Batch returned rows: 316
Sample results (first 3): [{'paper_id': '16b768df753f1bf5', 'llm_summary': 'This paper investigates the presence of low-dimensional chaos in the dry bulk shipping market using the Baltic Dry Index. While evidence suggests a nonlinear structure, chaotic and deterministic behavior could not be confirmed. The study found that GARCH and EGARCH models explain a significant portion of the nonlinearity.', 'i1_hint': True, 'i2_hint': False, 'i3_hint': True, 'model': 'gemini-2.5-flash-lite'}, {'paper_id': '32873e86106b0a11', 'llm_summary': 'This article presents a method for exact likelihood inference in a class of non-Gaussian state space models, applicable to models with stochastic intensity, volatility, or duration. It also details procedures for filtering and smoothing to estimate latent variables, suitable for both Bayesian and frequentist e

In [29]:
# Analysis: LLM Enhancement Quality Check

# Define criteria text if not already available
if 'CRITERIA_TEXT' not in globals():
    CRITERIA_TEXT = {
        "I1": "The article includes an empirical Machine Learning application (not pure theory). It goes beyond simple linear models.",
        "I2": "The article is about government/sovereign bonds and/or the yield curve",
        "I3": "The article is about forecasting or prediction",
    }

print("=" * 60)
print("📊 LLM ENHANCEMENT QUALITY ANALYSIS")
print("=" * 60)

# Check if enhanced CSV exists
if 'ENHANCED_CSV' not in globals():
    ENHANCED_CSV = Path(".") / "articles_with_gpt_response.csv"

if not ENHANCED_CSV.exists():
    print(f"❌ Enhanced dataset not found at {ENHANCED_CSV}")
    print("   Please run the LLM enhancement cell first!")
else:
    # Load the enhanced dataset
    enhanced_df = pd.read_csv(ENHANCED_CSV)
    
    print(f"\n📈 Dataset Overview:")
    print(f"   Total articles: {len(enhanced_df)}")
    
    # Check summaries
    print(f"\n📝 Summary Quality:")
    empty_summaries = enhanced_df[enhanced_df['llm_summary'].isna() | (enhanced_df['llm_summary'].astype(str).str.len() == 0)]
    error_summaries = enhanced_df[enhanced_df['llm_summary'].astype(str).str.contains('LLM parse/error', na=False)]
    print(f"   ✅ Valid summaries: {len(enhanced_df) - len(empty_summaries) - len(error_summaries)}")
    print(f"   ❌ Empty summaries: {len(empty_summaries)}")
    print(f"   ⚠️  Error summaries: {len(error_summaries)}")
    
    # Check inclusion criteria
    print(f"\n🔍 Inclusion Criteria Coverage:")
    for i in range(1, 4):
        col = f'i{i}_hint'
        if col in enhanced_df.columns:
            total = len(enhanced_df)
            none_count = enhanced_df[col].isna().sum()
            true_count = (enhanced_df[col] == True).sum()
            false_count = (enhanced_df[col] == False).sum()
            
            print(f"\n   I{i} ({CRITERIA_TEXT[f'I{i}'][:50]}...):")
            print(f"      ✓ True: {true_count} ({true_count/total*100:.1f}%)")
            print(f"      ✗ False: {false_count} ({false_count/total*100:.1f}%)")
            print(f"      ? None/Missing: {none_count} ({none_count/total*100:.1f}%)")
    
    # Overall quality score
    print(f"\n🎯 Overall Quality Score:")
    complete_rows = enhanced_df[
        (enhanced_df['llm_summary'].notna()) & 
        (enhanced_df['llm_summary'].astype(str).str.len() > 0) &
        (~enhanced_df['llm_summary'].astype(str).str.contains('LLM parse/error', na=False)) &
        (enhanced_df['i1_hint'].notna()) &
        (enhanced_df['i2_hint'].notna()) &
        (enhanced_df['i3_hint'].notna())
    ]
    quality_score = len(complete_rows) / len(enhanced_df) * 100
    print(f"   Complete records (all fields valid): {len(complete_rows)}/{len(enhanced_df)} ({quality_score:.1f}%)")
    
    if quality_score < 100:
        print(f"\n⚠️  Warning: {len(enhanced_df) - len(complete_rows)} articles have incomplete data")
        incomplete = enhanced_df[~enhanced_df.index.isin(complete_rows.index)]
        print(f"   Sample incomplete records:")
        for idx, row in incomplete.head(3).iterrows():
            issues = []
            if pd.isna(row['llm_summary']) or len(str(row['llm_summary'])) == 0:
                issues.append("no summary")
            if 'LLM parse/error' in str(row['llm_summary']):
                issues.append("error summary")
            if pd.isna(row['i1_hint']):
                issues.append("I1 missing")
            if pd.isna(row['i2_hint']):
                issues.append("I2 missing")
            if pd.isna(row['i3_hint']):
                issues.append("I3 missing")
            print(f"      - {row['paper_id']}: {', '.join(issues)}")
    
    print("\n" + "=" * 60)


📊 LLM ENHANCEMENT QUALITY ANALYSIS

📈 Dataset Overview:
   Total articles: 1203

📝 Summary Quality:
   ✅ Valid summaries: 1203
   ❌ Empty summaries: 0
   ⚠️  Error summaries: 0

🔍 Inclusion Criteria Coverage:

   I1 (The article includes an empirical Machine Learning...):
      ✓ True: 838 (69.7%)
      ✗ False: 365 (30.3%)
      ? None/Missing: 0 (0.0%)

   I2 (The article is about government/sovereign bonds an...):
      ✓ True: 360 (29.9%)
      ✗ False: 843 (70.1%)
      ? None/Missing: 0 (0.0%)

   I3 (The article is about forecasting or prediction...):
      ✓ True: 875 (72.7%)
      ✗ False: 328 (27.3%)
      ? None/Missing: 0 (0.0%)

🎯 Overall Quality Score:
   Complete records (all fields valid): 1203/1203 (100.0%)



In [30]:
# Reviewer assignment (reproducible, saved for screening)
import numpy as np

REVIEWERS = ["Olav", "Ulrik", "Trine"]
SEED = int(os.environ.get("SCREENING_SEED", 42))
ASSIGNMENTS_CSV = OUT_DIR / "outputs/assignments.csv"
ASSIGNMENTS_CSV.parent.mkdir(parents=True, exist_ok=True)

assignments = final_df[["paper_id", "title", "year", "doi"]].copy()
reviewers_cycle = (REVIEWERS * ((len(assignments) + len(REVIEWERS) - 1) // len(REVIEWERS)))[:len(assignments)]
np.random.seed(SEED)
np.random.shuffle(reviewers_cycle)
assignments["reviewer"] = reviewers_cycle
assignments.to_csv(ASSIGNMENTS_CSV, index=False)
print("Assignments saved:", ASSIGNMENTS_CSV, "rows:", len(assignments))


Assignments saved: outputs/assignments.csv rows: 1203


We duplicate the generated assignments.csv document and randomize the assignment to another reviewer to facilitate for dual initial screening of each paper:

In [18]:
import pandas as pd
import numpy as np
import os

ASSIGNMENTS_IN = r"..\Screening_1\outputs\assignments.csv"
ASSIGNMENTS_OUT = r"..\Screening_1\outputs\initial_screening_2\assignments2.csv"
assignments = pd.read_csv(ASSIGNMENTS_IN)

REVIEWERS = ["Olav", "Ulrik", "Trine"]
SEED = int(os.environ.get("SCREENING_SEED", 42))
np.random.seed(SEED)

n = len(assignments)
base = n // len(REVIEWERS)
remainder = n % len(REVIEWERS)
counts = [base + (1 if i < remainder else 0) for i in range(len(REVIEWERS))]

# build balanced pool and try to shuffle into a derangement (no fixed points)
candidates = []
for reviewer, cnt in zip(REVIEWERS, counts):
    candidates += [reviewer] * cnt

max_tries = 2000
for _ in range(max_tries):
    np.random.shuffle(candidates)
    if all(c != r for c, r in zip(candidates, assignments["reviewer"].astype(str))):
        break
else:
    # greedy fallback swaps to resolve remaining conflicts
    for i, (orig, cand) in enumerate(zip(assignments["reviewer"].astype(str), candidates)):
        if orig == cand:
            for j in range(n):
                if i == j:
                    continue
                if candidates[j] != orig and candidates[i] != assignments.loc[j, "reviewer"]:
                    candidates[i], candidates[j] = candidates[j], candidates[i]
                    break

# final safety: force any remaining conflicts to a different reviewer
for i, orig in enumerate(assignments["reviewer"].astype(str)):
    if candidates[i] == orig:
        for r in REVIEWERS:
            if r != orig:
                candidates[i] = r
                break

# overwrite the assignments column
assignments["reviewer"] = candidates
assignments[["paper_id", "title", "year", "doi", "reviewer"]].to_csv(ASSIGNMENTS_OUT, index=False)